In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 11-2. NLsolve.jl による非線形方程式の求解

※ 本文とは環境等を少し変えています（コードはほぼ同一です）

In [2]:
]activate NLSolveSample

  Activating new project at `/path/to/JuliaBook2023Samples/notebooks/Chapter11/NLSolveSample`


### コード11-4. `NLsolve.jl` パッケージの追加

```julia
julia> # `]` をタイプしてパッケージモードに移行

(@v1.7) pkg> add NLsolve
    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General`
   Resolving package versions...
    Updating `~/.julia/environments/v1.7/Project.toml`
  [2774e3e8] + NLsolve v4.5.1
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [4fba245c] + ArrayInterface v5.0.5
  [d360d2e6] + ChainRulesCore v1.14.0
   ：《以下略》

(@v1.7) pkg> st NLsolve
      Status `~/.julia/environments/v1.7/Project.toml`
  [2774e3e8] NLsolve v4.5.1

(@v1.7) pkg> st -m
      Status `~/.julia/environments/v1.7/Manifest.toml`
   ：《中略》
  [6a86dc24] FiniteDiff v2.11.0
  [f6369f11] ForwardDiff v0.10.25
   ：《中略》
  [d41bc354] NLSolversBase v7.8.2
  [2774e3e8] NLsolve v4.5.1
   ：《以下略》
```

In [3]:
]add NLsolve@4.5.1

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `git@github.com:JuliaRegistries/General.git`
   Resolving package versions...
    Updating `/path/to/JuliaBook2023Samples/notebooks/Chapter11/NLSolveSample/Project.toml`
  [2774e3e8] + NLsolve v4.5.1
    Updating `/path/to/JuliaBook2023Samples/notebooks/Chapter11/NLSolveSample/Manifest.toml`
  [30b0a656] + ArrayInterfaceCore v0.1.29
  [d360d2e6] + ChainRulesCore v1.15.7
  [9e997f8a] + ChangesOfVariables v0.1.5
  [bbf7d656] + CommonSubexpressions v0.3.0
  [34da2185] + Compat v4.5.0
  [187b0558] + ConstructionBase v1.4.1
  [163ba53b] + DiffResults v1.1.0
  [b552c78f] + DiffRules v1.12.2
  [b4f34e82] + Distances v0.10.7
  [ffbed154] + DocStringExtensions v0.9.3
  [6a86dc24] + FiniteDiff v2.17.0
  [f6369f11] + ForwardDiff v0.10.34
  [3587e190] + InverseFunctions v0.1.8
  [92d709cd] + IrrationalConstants v0.1.1
  [692b3bcd] + JLLWrappers v1.4.1
  [d3d80556] + LineSearches v7.2.0
  [2ab3a3ac] + LogExpFunctions v0.3.

In [4]:
]st NLsolve

Status `/path/to/JuliaBook2023Samples/notebooks/Chapter11/NLSolveSample/Project.toml`
  [2774e3e8] NLsolve v4.5.1


In [5]:
]st -m

Status `/path/to/JuliaBook2023Samples/notebooks/Chapter11/NLSolveSample/Manifest.toml`
  [30b0a656] ArrayInterfaceCore v0.1.29
  [d360d2e6] ChainRulesCore v1.15.7
  [9e997f8a] ChangesOfVariables v0.1.5
  [bbf7d656] CommonSubexpressions v0.3.0
  [34da2185] Compat v4.5.0
  [187b0558] ConstructionBase v1.4.1
  [163ba53b] DiffResults v1.1.0
  [b552c78f] DiffRules v1.12.2
  [b4f34e82] Distances v0.10.7
  [ffbed154] DocStringExtensions v0.9.3
  [6a86dc24] FiniteDiff v2.17.0
  [f6369f11] ForwardDiff v0.10.34
  [3587e190] InverseFunctions v0.1.8
  [92d709cd] IrrationalConstants v0.1.1
  [692b3bcd] JLLWrappers v1.4.1
  [d3d80556] LineSearches v7.2.0
  [2ab3a3ac] LogExpFunctions v0.3.20
  [1914dd2f] MacroTools v0.5.10
  [d41bc354] NLSolversBase v7.8.3
  [2774e3e8] NLsolve v4.5.1
  [77ba4419] NaNMath v1.0.1
  [bac558e1] OrderedCollections v1.4.1
  [d96e819e] Parameters v0.12.3
  [21216c6a] Preferences v1.3.0
  [189a3867] Reexport v1.2.2
  [ae029012] Requires v1.3.0
  [efcf1570] Setfield v1.1.1
  

$$
\begin{cases}
\begin{aligned}
u(x, y) &= (x + 3)(y^3-7)+18 \\
v(x, y) &= \sin(y \cdot \mathrm{e}^x-1)
\end{aligned}
\end{cases}
$$

### コード11-5. `NLsolve.jl` の使用例(1) （README に記載のサンプル）

In [6]:
using NLsolve

In [7]:
function f!(F, xy)
    x, y = xy
    F[1] = (x+3)*(y^3-7)+18  # u
    F[2] = sin(y*exp(x)-1)  # v
end

f! (generic function with 1 method)

In [8]:
function j!(J, xy)
    x, y = xy
    J[1, 1] = y^3-7  # ∂u/∂x
    J[1, 2] = 3y^2*(x+3)  # ∂u/∂y
    ∂v_∂y = exp(x)*cos(y*exp(x)-1)  # ∂v/∂y
    J[2, 1] = y*∂v_∂y  # ∂v/∂x
    J[2, 2] = ∂v_∂y
end

j! (generic function with 1 method)

In [9]:
sol_uv = nlsolve(f!, j!, [0.1, 1.2])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.1, 1.2]
 * Zero: [-3.7818049096324184e-16, 1.0000000000000002]
 * Inf-norm of residuals: 0.000000
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 5
 * Jacobian Calls (df/dx): 5

$$
J = \begin{bmatrix}
\begin{aligned}\frac{\partial u}{\partial x}\end{aligned} & \begin{aligned}\frac{\partial u}{\partial y}\end{aligned} \\
\begin{aligned}\frac{\partial u}{\partial x}\end{aligned} & \begin{aligned}\frac{\partial u}{\partial y}\end{aligned} \\
\end{bmatrix} = \left [\ \begin{array}{ll}
y^3-7 & 3y^2 \ (x+3) \\
y \ \mathrm{e}^x \cos(y \ \mathrm{e}^x-1) & \mathrm{e}^x \cos(y \ \mathrm{e}^x-1)
\end{array} \right ]
$$

### コード11-6. 求解結果の確認

In [10]:
sol_uv.f_converged

true

In [11]:
sol_uv.zero ≈ [0.0, 1.0]

true

In [12]:
res = similar(sol_uv.zero, 2); f!(res, sol_uv.zero);

In [13]:
# res ≈ [0.0, 0.0]  # ←NG!
≈(res, [0.0, 0.0]; atol=1e-8)

true

### コード11-7. `NLsolve.jl` の使用例(2) （自動微分を利用する場合）

In [14]:
# `j!` 指定しなくても求解可能（内部で自動微分が実施される）
nlsolve(f!, [0.1, 1.2])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.1, 1.2]
 * Zero: [-7.775548712324193e-17, 0.9999999999999999]
 * Inf-norm of residuals: 0.000000
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 5
 * Jacobian Calls (df/dx): 5

In [15]:
# `ForwardDiff` 利用を指定
nlsolve(f!, [0.1, 1.2]; autodiff=:forward)

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.1, 1.2]
 * Zero: [-3.7818049096324184e-16, 1.0000000000000002]
 * Inf-norm of residuals: 0.000000
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 5
 * Jacobian Calls (df/dx): 5

### コード11-8. `NLsolve.jl` の使用例(3) （解法を指定）

In [16]:
# 自動微分に `ForwardDiff` 利用を指定、解法に :newton を指定
nlsolve(f!, [0.1, 1.2]; autodiff=:forward, method=:newton)

Results of Nonlinear Solver Algorithm
 * Algorithm: Newton with line-search
 * Starting Point: [0.1, 1.2]
 * Zero: [-3.7818049096324184e-16, 1.0000000000000002]
 * Inf-norm of residuals: 0.000000
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 5
 * Jacobian Calls (df/dx): 5

In [17]:
# 解法に :broyden を指定（!j 不要）
nlsolve(f!, [0.1, 1.2]; method=:broyden)

Results of Nonlinear Solver Algorithm
 * Algorithm: broyden without line-search
 * Starting Point: [0.1, 1.2]
 * Zero: [7.681043703429256e-12, 1.0000000000322504]
 * Inf-norm of residuals: 0.000000
 * Iterations: 8
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 41
 * Jacobian Calls (df/dx): 0

In [18]:
# 解法に :anderson を指定（!j 不要）
nlsolve(f!, [0.1, 1.2]; method=:anderson)

Results of Nonlinear Solver Algorithm
 * Algorithm: Anderson m=2 beta=1 aa_start=1 droptol=1.0e10
 * Starting Point: [0.1, 1.2]
 * Zero: [6.546901238425849e-13, 0.9999999999970071]
 * Inf-norm of residuals: 0.000000
 * Iterations: 29
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 29
 * Jacobian Calls (df/dx): 0